This notebook attempts to predict the effectiveness of different types of apps for epidemiological contact tracing. Initial (and still very rough) version by Peter Eckersley <peter.eckersley@gmail.com> and Lewis Mitchell <lewis.mitchell@adelaide.edu.au>; feel free to reuse and repurpose, but please indicate that (as of this version) results are not peer reviewed, and do not imply endorsement of any conclusion or policy!


In [0]:
import numpy as np
import numpy.random as npr
import bokeh.plotting as bp
import bokeh.io as bi
import bokeh.util.hex as bh
import bokeh.transform as bt
import bokeh.models as bm
import bokeh.colors as bc
import bokeh.layouts as bl
import matplotlib.pyplot as plt
from scipy.integrate import odeint

import requests

**Things that are done**:

*   Monte Carlo simulation of app adoption & effectiveness for two types of app:
  * A "bluetooth" model that is more accurate, but requires both infected and exposed individuals to have the app installed in order to anonymously measure exposure using a [protocol like this one](https://docs.google.com/document/d/1f65V3PI214-uYfZLUZtm55kdVwoazIMqGJrxcYNI4eg/edit#heading=h.6q40wl39kcs8).
  * A less accurate retrospective mobile location (GPS+Wifi) model that assumes that when patients are diagnosed, they can be sent an onboarding link which uses their Google Maps Timeline or iOS on-device location records to identify (using [private set intersection](https://en.wikipedia.org/wiki/Private_set_intersection) or similar methods) and send them notifications. For the model, the important point is that patients do not need to have had the app installed at the time of contact.
*   Differential equation model of pandemic for each simulation, including:
  * A lockdown policy that kicks in during periods of high infection
  * App launching at some date (Monte Carlo distributed) that diverts some exposed individuals into a quarantine state that's separate from the usual exposed -> infected -> recovered path and does not cause further transmission
*   Analysis of impact of the app on fatalities and lockdown duration, as a function of adoption

**Things to definitely do**:
* Test SQUEE step size + accuracy
* Estimate lives saved & days of lockdown averted for the retrospective GPS model
* Ground policy intervention effect sizes better
   * Social distancing from observational data?
   * And/or from [other modelling approaches](https://www.sciencedirect.com/science/article/pii/S1473309920301444) 
* Discount the effect of contact tracing for the proportion of traces that arrive too late
* Graph against every MC variable, and compute partially ranked correlation coefficients
* Set a "model done" deadline, and pivot to paper at that point
    * Thursday morning!


**Things to consider doing**:
* SEEIIR?
* Model eradication policies?
* Consider modelling false positive alerts explicitly
* Model household transmission, which continues after some forms of isolation into state Q
* Adjust the lockdown model to be non-binary. Presently the model turns lockdowns on and off more frequently than is realistic, since these policies are inherently slow to stop and start (though it may be reasonable to view the total number of days of binary lockdown as an approximation to the effect and burden of a more nuanced and slow-changing set of policies)

In [0]:
def franken_dist(mean, high95, low95, size=1):
    "Create an asymmetrically stretched normal distribution"
    dist = npr.normal(size=size)
    dist = np.where(dist < 0, dist * low95 / 2, dist * high95 / 2)
    dist += mean
    return dist

In [0]:
# we use a Monte Carlo approach to handle uncertainty in parameters; we call down to an SEIRQL 
# differential equation pandemic model to estimate # of lives saved
samples = 10000
npr.seed(0)

# model the United States
N = 327 * 10**6
# start date is beginning of February 2020

# Variables for our model

data_raw = dict(
  # These are for the app
  pop_adoption = npr.uniform(0, 1.0, size=samples),  # independent variable, explore all adoption levels
  #pop = 1 / npr.pareto(1e6, size=samples),  # scale of population, roughly cities to countries
  tester_adoption = npr.uniform(0.3, 1.0, size=samples),  # when a test is positive, does that wind up in the app?
  #testing_rate = np.clip(npr.pareto(1, size=samples), 0, 1),  # XXX BUGGY Fraction of infections that get tested. Typical range from 0.001 (US or Indonesia) to above 0.5 (Singapore)
  testing_rate = franken_dist(0.12, 0.15, 0.022, samples),  # approximate https://cmmid.github.io/topics/covid19/severity/global_cfr_estimates.html#current-estimates US numbers
  catch_rate_bt = npr.uniform(0.4, 0.8, size=samples),  # how often does the software detect the contact between two of its users XXX needs more modelling
  catch_rate_gps = npr.uniform(0.3, 0.7, size=samples),  # assume GPS is less precise and might fail worse, though best case is as good because of intertemporal fomite risk detection
)

pandemic_params = dict(
  # these are for the pandemic
  r0_raw = (1.6 + npr.beta(2, 5, size=samples) * 3),  # TODO: ground better in https://github.com/midas-network/COVID-19/tree/master/parameter_estimates/2019_novel_coronavirus#basic-reproduction-number
  #r0 = npr.uniform(1.5, 3.5, size=samples),
  #app_launch_date = npr.randint(50, 120, size=samples),  # days from 2020-02-01
  app_launch_date = npr.randint(65, 90, size=samples),
  #infection_fatality_rate = npr.uniform(0.11, 4.3, size=samples)/100,  # follow https://www.medrxiv.org/content/medrxiv/early/2020/03/09/2020.03.05.20031773.full.pdf
                                                                       # but err a little higher due to subsequent Diamond Princess deaths & ICU overload risks
  infection_fatality_rate = npr.beta(2, 5, size=samples)*4./100,
  # could use npr.lognormal(-0.1, 0.9, size=samples)
  #lockdown_threshold = 1 / npr.pareto(100, size=samples),  # 99% CI~ 20-10,000 new cases per day for lockdown
  lockdown_threshold = 1 / npr.pareto(200, size=samples),
  #lockdown_effect = npr.uniform(0.2, 0.9, size=samples),
  lockdown_r0 = npr.uniform(0.4, 1.3, size=samples),  # lower bound from Australian estimates, upper bound is a somewhat unsuccessful effort

  #lockdown_limit = npr.randint(20, 200, size=samples)  # hard limit before politics or economics makes lockdown impossible
  lockdown_limit = npr.lognormal(4, size=samples)
  #lockdown_limit = npr.randint(1000,2000, size=samples)
)
data_raw.update(pandemic_params)
data_raw["tester_adoption"] = np.minimum(data_raw["tester_adoption"], data_raw["pop_adoption"])  # assume that if you have very high population adoption, testers probably use this too
globals().update(data_raw)  # refactorme out
data = bm.ColumnDataSource(data_raw)

In [0]:
def display(plot):
  bi.curdoc().add_root(plot)
  bp.output_notebook()
  bi.show(plot)

In [84]:
import datetime as dt
import requests
import json

j = requests.get("https://covidtracking.com/api/us/daily").json()
print(json.dumps(j[:2], indent=4))

[
    {
        "date": 20200411,
        "states": 56,
        "positive": 522843,
        "negative": 2142823,
        "pending": 16593,
        "hospitalizedCurrently": 51409,
        "hospitalizedCumulative": 51114,
        "inIcuCurrently": 13563,
        "inIcuCumulative": 1228,
        "onVentilatorCurrently": 5978,
        "onVentilatorCumulative": 41,
        "recovered": 24196,
        "hash": "479e0eb862cd6da03ef3d7e41fa285fb733a7f64",
        "dateChecked": "2020-04-11T20:00:00Z",
        "death": 20355,
        "hospitalized": 51114,
        "total": 2682259,
        "totalTestResults": 2665666,
        "posNeg": 2665666,
        "deathIncrease": 1867,
        "hospitalizedIncrease": 431,
        "negativeIncrease": 106793,
        "positiveIncrease": 29591,
        "totalTestResultsIncrease": 136384
    },
    {
        "date": 20200410,
        "states": 56,
        "positive": 493252,
        "negative": 2036030,
        "pending": 17435,
        "hospitalizedCurrently"

In [93]:

testmap = np.zeros(365)
last = 0
for entry in j:
    date = dt.datetime.strptime(str(entry["date"]), "%Y%m%d")
    start = dt.datetime(2020, 2, 1)
    day = (date - start).days
    if day > last: last = day
    if day >= 0:
        testmap[day] = entry["totalTestResultsIncrease"]

plot = bp.figure(plot_width=768, plot_height=768, y_axis_type="linear",
                 x_axis_label="Day", y_axis_label="Tests in the US")

plot.scatter(t[:last], testmap[:last])
fitstart = last - 5
nppp = np.polynomial.polynomial  # really!
npppp = nppp.Polynomial
extrapolation = nppp.polyfit(t[fitstart:last], testmap[fitstart:last], 1)
p = np.polynomial.Polynomial(extrapolation)
testmap[last:365] = p(t[last:365])
plot.line(t[fitstart:fitstart+60], p(t[fitstart:fitstart+60]), legend_label="extrapolation", line_color="red")
#p(np.arange(fitstart,365)))
display(plot)

In [63]:
print(extrapolation)
more = nppp.polyval(t[last:365], extrapolation)
print(more)

poly([41885826.60908861 60050107.07575373 19385846.21211966])
[Polynomial([41885826.60908861, 60050107.07575373, 19385846.21211966], domain=[ 60., 365.], window=[-1.,  1.])
 Polynomial([41885826.60908861, 60050107.07575373, 19385846.21211966], domain=[ 60., 365.], window=[-1.,  1.])
 Polynomial([41885826.60908861, 60050107.07575373, 19385846.21211966], domain=[ 60., 365.], window=[-1.,  1.])
 Polynomial([41885826.60908861, 60050107.07575373, 19385846.21211966], domain=[ 60., 365.], window=[-1.,  1.])
 Polynomial([41885826.60908861, 60050107.07575373, 19385846.21211966], domain=[ 60., 365.], window=[-1.,  1.])
 Polynomial([41885826.60908861, 60050107.07575373, 19385846.21211966], domain=[ 60., 365.], window=[-1.,  1.])
 Polynomial([41885826.60908861, 60050107.07575373, 19385846.21211966], domain=[ 60., 365.], window=[-1.,  1.])
 Polynomial([41885826.60908861, 60050107.07575373, 19385846.21211966], domain=[ 60., 365.], window=[-1.,  1.])
 Polynomial([41885826.60908861, 60050107.07575373,

In [0]:
# summary of each simulation for the tooltip
tooltips_bt = [
    ("societal adoption", "@pop_adoption"),
    ("tester adoption", "@tester_adoption"),
    ("testing rate", "@testing_rate"),
    ("app contact detection rate", "@catch_rate_bt"),  # needs to change
    ("trace success rate", "$y")
]

tooltips_gps = tooltips_bt[:]
tooltips_gps[-2] = ("app contact detection rate", "@catch_rate_gps")

In [6]:
def coverage1(adoption, testing_rate, tester_adoption, catch_rate):
  """
  Tramsmission event coverage for an app that needs to be on both users' phones, at the time of
  exposure, eg by bluetooth matching. *NOTE* this assumes iOS and Android can see each other.
  If not, coverage is roughly halved :(
  """
  intersection = adoption * adoption
  return intersection * testing_rate * tester_adoption * catch_rate

cov_bt = coverage1(pop_adoption, testing_rate, tester_adoption, catch_rate_bt)
data.add(cov_bt, "cov_bt")

'cov_bt'

In [0]:
step = 0.03
incs = np.arange(0, 1, step)

def bin_stats(variable, statistic, group_by=pop_adoption):
  bins = [[] for n in incs]

  for adoption, v in zip(group_by, variable):
    i = int(adoption // step)
    bins[i].append(v)

  binned = np.array([statistic(b) for b in bins])
  return binned

bin_averages = lambda data: bin_stats(data, np.average)
bin_25 = lambda data: bin_stats(data, lambda x: np.quantile(x, 0.25))
bin_75 = lambda data: bin_stats(data, lambda x: np.quantile(x, 0.75))

avgs = bin_averages(cov_bt)

Now compare to an app where the assumption is that diagnosed patients contribute to an anonymous redzone map based on retrospective location history

In [0]:

onboarding_loss = npr.uniform(0.1, 0.6, size=samples)  # patients who refuse or fail to install the app at diagnosis time
onboarding_loss = np.minimum(onboarding_loss, 1 - pop_adoption)  # but if 100% of users have the app, diagnosed patients do too

def coverage2(adoption, testing_rate, tester_adoption, catch_rate2):
  return adoption * testing_rate * tester_adoption * catch_rate2 * (1 - onboarding_loss)

cov_gps = coverage2(pop_adoption, testing_rate, tester_adoption, catch_rate_gps)
data.add(cov_gps, "cov_gps")
avgs2 = bin_averages(cov_gps)

In [0]:
plot = bp.figure(x_range=[0,1], y_range=[0, 1], plot_width=768, plot_height=768,
                 x_axis_label="Proportion of population using app",
                 y_axis_label="Proportion of infections traced",
                 #,tooltips=tooltips_bt
                 )
plot.xaxis.axis_label_text_font_size = plot.yaxis.axis_label_text_font_size="16pt"
plot.xaxis.major_label_text_font_size = plot.yaxis.major_label_text_font_size ="11pt"

plot.scatter(x="pop_adoption", y="cov_bt", radius=0.005, fill_alpha=0.25, line_color=None, 
             source=data, legend_label="bluetooth simulation")
plot.scatter(pop_adoption, cov_gps, radius=0.005, fill_alpha=0.15, line_color=None,
             fill_color="#209000", legend_label="gps simulation")
plot.line(incs, bin_25(cov_bt), line_color="#0000a0", legend_label="bluetooth +/- 25%", line_width=1.4)
plot.line(incs, bin_75(cov_bt), line_color="#0000a0", line_width=1.4)

plot.line(incs, avgs2, line_color="#505000", legend_label="retrospective gps matching (mean)", line_width=2.0)
plot.line(incs, bin_25(cov_gps), line_color="#80f080", legend_label="gps +/- 25%", line_width=1.4)
plot.line(incs, bin_75(cov_gps), line_color="#80f080", line_width=1.4)

plot.line(incs, avgs, line_color="#a000f0", legend_label="prospective bluetooth matching (mean)", line_width=2.0)  
plot.legend.label_text_font_size = '11pt'

bi.curdoc().add_root(plot)
bp.output_notebook()
bi.show(plot)
print("Monte Carlo simulation of "
      "effectiveness as a function of adoption for bluetooth matching apps (blue) and apps that\n"
      "use retrospective location records such as Google Maps Timeline or iOS's on-device encrypted\n"
      "location records (green).")

## (Code below this point is experimental)

In [0]:
import numpy as np
import numpy.random as npr
import matplotlib.pyplot as plt
# import EoN
from scipy.integrate import odeint

In [0]:
def human_format(num):
    num = float('{:.3g}'.format(num))
    magnitude = 0
    while abs(num) >= 1000:
        magnitude += 1
        num /= 1000.0
    return '{}{}'.format('{:f}'.format(num).rstrip('0').rstrip('.'), ['', 'K', 'M', 'B', 'T'][magnitude])

In [0]:
# adapted from https://scipython.com/book/chapter-8-scipy/additional-examples/the-sir-epidemic-model/
t = np.linspace(0, 364, 365)

# adapted from https://scipython.com/book/chapter-8-scipy/additional-examples/the-sir-epidemic-model/
t = np.linspace(0, 364, 365)
def simulate_quarantined_epidemic2(r0, contact_tracing_rate, launch_date, lockdown_threshold,
                                   lockdown_r0, lockdown_limit):
  "Run a simple SEIQR model of an epidemic"
  # solved for infection_rate using https://science.sciencemag.org/content/early/2020/03/24/science.abb3221.full
  infect_rate, recovery_rate = 1./3, 1./2.5
  # Total population, N.
  #N = 25*10**6
  # Initial number of infected and recovered individuals, I0 and R0.
  E0, I0, R_init, Q0 = 100, 0, 0, 0
  L0 = 0.  # days of lockdown
  # Everyone else, S0, is susceptible to infection initially.
  S0 = N - I0 - R_init - E0 - Q0
  # Contact rate, beta, and mean recovery rate, gamma, (in 1/days).
  # A grid of time points (in days)

  global r0_records
  r0_records = {}
  global lockdown_map

  lockdown_map = np.zeros(365 + 500)  # allow odeint to play outside the domain on the right

  # The SEEIIR model differential equations.
  def deriv(y, t, N, base_r0, infect_rate, recovery_rate, contact_tracing_rate, launch_date,
            lockdown_threshold, lockdown_r0, lockdown_limit):
      S, E, I, R, Q, L = y

      global lockdown_map, r0_records
      #assert t > ratchet, "deriv not called in order"
      #if I / N > lockdown_threshold:
      incidence = infect_rate * E
      if incidence > lockdown_threshold and L <= lockdown_limit:
          r0 = lockdown_r0
          dLdt = 1.
          if t > 0:
              length = int(np.clip(np.clip(7, None, lockdown_limit - L), 0, None))
              if length > 0:
                  lockdown_map[int(t):int(t) + length] = np.ones(length)
              lockdown_map[int(t):int(t) + length] = np.ones(length)
      elif lockdown_map[int(t)]:   #
          r0 = lockdown_r0
          dLdt = 1.
      else:
          r0 = base_r0
          dLdt = 0.
      r0_records[t] = r0

      contact_rate = recovery_rate * r0

      contact_tracing_rate = contact_tracing_rate * np.heaviside(t - app_launch_date, 1)

      dSdt = -contact_rate * S * I / N
      dEdt = (1 - contact_tracing_rate) * contact_rate * S * I / N - incidence
      dIdt = incidence - recovery_rate*I
      dRdt = recovery_rate * I
      # Q represents people who are quarantined *due to the app intervention* ; other
      # types of qurantine should be reflected in the value of R0
      dQdt = contact_tracing_rate*contact_rate * S * I / N
      return dSdt, dEdt, dIdt, dRdt, dQdt, dLdt

  # Initial conditions vector
  y0 = S0, E0, I0, R_init, Q0, L0
  # Integrate the SIR equations over the time grid, t.
  print("odeint call", (N, r0, infect_rate, recovery_rate, contact_tracing_rate, launch_date,
                                   lockdown_threshold, lockdown_r0, lockdown_limit))
  ret = odeint(deriv, y0, t, args=(N, r0, infect_rate, recovery_rate, contact_tracing_rate, launch_date,
                                   lockdown_threshold, lockdown_r0, lockdown_limit))#, rtol=1e-12, atol=1e-12)
  # S, E, I, R, Q, L = ret.T
  return ret.T, r0_records



def simulate_quarantined_epidemic_euler(r0, contact_tracing_rate, launch_date, lockdown_threshold,
                                        lockdown_r0, lockdown_limit):
  "Run a simple SEIQR model of an epidemic using a friggin simple Euler method, except it's actually a RK4 scheme!"
  # solved for infection_rate using https://science.sciencemag.org/content/early/2020/03/24/science.abb3221.full
  # recovery rate from https://www.doherty.edu.au/uploads/content_doc/McVernon_Modelling_COVID-19_07Apr1_with_appendix.pdf
  infect_rate, recovery_rate = 1./3, 1./5
  # Total population, N.
  #N = 25*10**6
  # Initial number of infected and recovered individuals, I0 and R0.
  E10, E20, I10, I20, R_init, Q0 = 100, 0, 0, 0, 0, 0
  L0 = 0.  # days of lockdown
  # Everyone else, S0, is susceptible to infection initially.
  S0 = N - I10 - I20 - R_init - E10 - E20 - Q0
  # Contact rate, beta, and mean recovery rate, gamma, (in 1/days).
  # A grid of time points (in days)


  global r0_records, lockdown_map
  r0_records = {}
  lockdown_map = np.zeros(365+7)
  

  # The SEEIIR model differential equations.
  def deriv(y, t, N, base_r0, infect_rate, recovery_rate, contact_tracing_rate, launch_date,
            lockdown_threshold, lockdown_r0, lockdown_limit):
      S, E1, E2, I1, I2, R, Q, L = y

      I =  I1 + I2
      global lockdown_map, ratchet, r0_records
      # assert t > ratchet, "deriv not called in order"
      incidence = 2*infect_rate * E2

      if incidence > lockdown_threshold and L <= lockdown_limit:
          r0 = base_r0 * lockdown_r0
          dLdt = 1.
          if t > 0:
              length = int(np.clip(np.clip(7, None, lockdown_limit - L), 0, None))
              if length > 0:
                  lockdown_map[int(t):int(t) + length] = np.ones(length)
              lockdown_map[int(t):int(t) + length] = np.ones(length)
      elif lockdown_map[int(t)]:   #
          r0 = base_r0 * lockdown_r0
          dLdt = 1.
      else:
          r0 = base_r0
          dLdt = 0.
      r0_records[t] = r0

      contact_rate = recovery_rate * r0
      contact_tracing_rate = contact_tracing_rate * np.heaviside(t - launch_date, 1)

      dSdt = -contact_rate * S * I / N
      dE1dt = (1 - contact_tracing_rate) * contact_rate * S * I / N - 2*infect_rate*E1
      dE2dt = 2*infect_rate*E1 - incidence
      dI1dt = incidence - 2*recovery_rate*I1
      dI2dt = 2*recovery_rate*(I1 - I2)
      dRdt = 2*recovery_rate * I2
      # Q represents people who are quarantined *due to the app intervention* ; other
      # types of qurantine should be reflected in the value of R0
      dQdt = contact_tracing_rate*contact_rate * S * I / N
      return dSdt, dE1dt, dE2dt, dI1dt, dI2dt, dRdt, dQdt, dLdt


  # Initial conditions vector
  y0 = S0, E10, E20, I10, I20, R_init, Q0, L0

  # Integrate the SIR equations over the time grid, t.
  # ret = odeint(deriv, y0, t, args=(N, r0, infect_rate, recovery_rate, contact_tracing_rate,
  #                                  lockdown_threshold, lockdown_r0))#, rtol=1e-12, atol=1e-12)
  ret = np.zeros((len(t),len(y0)))
  ret[0,:] = y0
  n_inner = 1 # number of steps in inner euler loop (reducing timestep by 1/n_inner)
  for i,ti in enumerate(t[1:]):
    dt = ti - t[i]
    ## Old-fashioned Euler scheme
    # ret[i+1,:] = ret[i,:] + dt*np.array([dxdt for dxdt in deriv(ret[i,:], ti, N, r0, 
    #                                                    infect_rate, recovery_rate, 
    #                                                    contact_tracing_rate, launch_date, lockdown_threshold, 
    #                                                    lockdown_r0, lockdown_limit)])
    
    ## Modern and totally sweet RK4 scheme!
    k1 = np.array([dxdt for dxdt in deriv(ret[i,:], ti, N, r0,
                                          infect_rate, recovery_rate,
                                          contact_tracing_rate, launch_date, lockdown_threshold,
                                          lockdown_r0, lockdown_limit)])
    k2 = np.array([dxdt for dxdt in deriv(ret[i,:] + dt*k1/2., ti + dt/2., N, r0,
                                          infect_rate, recovery_rate,
                                          contact_tracing_rate, launch_date, lockdown_threshold,
                                          lockdown_r0, lockdown_limit)])
    k3 = np.array([dxdt for dxdt in deriv(ret[i,:] + dt*k2/2., ti + dt/2., N, r0,
                                          infect_rate, recovery_rate,
                                          contact_tracing_rate, launch_date, lockdown_threshold,
                                          lockdown_r0, lockdown_limit)])
    k4 = np.array([dxdt for dxdt in deriv(ret[i,:] + dt*k3, ti + dt, N, r0,
                                          infect_rate, recovery_rate,
                                          contact_tracing_rate, launch_date, lockdown_threshold,
                                          lockdown_r0, lockdown_limit)])
    ret[i+1,:] = ret[i,:] + 1./6*dt*(k1 + 2*k2 + 2*k3 + k4)

  ret = np.array([ret[:,0],ret[:,1] + ret[:,2],ret[:,3] + ret[:,4],ret[:,5],ret[:,6],ret[:,7]]).T
  
  # S, E, I, R, Q, L = ret.T
  return ret.T, r0_records

squee = simulate_quarantined_epidemic_euler

In [0]:
# Explore how the app launch day affects dynamics

def display(plot):
  bi.curdoc().add_root(plot)
  bp.output_notebook()
  bi.show(plot)

test_solvers = False
if test_solvers:
    plot = bp.figure(plot_width=768, plot_height=400, y_axis_type="linear", y_range=[0, 0.1],
                    x_axis_label="Day", y_axis_label="Infected",
                    title="Illustrate the basic dynamics of the epidemic model for different app "
                          "launch dates (old vs new solver)")
    plot2 = bp.figure(plot_width=768, plot_height=400, y_axis_type="linear",
                    x_axis_label="Day", y_axis_label="Quarantined via app")

    test_r0 = 2.4

    color_mapper = bm.LinearColorMapper(palette="Viridis256", low=20, high=150)

    for launch_date in range(20, 150, 3):
      (S, E, I, R, Q, L), _r0r = simulate_quarantined_epidemic2(test_r0, 0.2, launch_date, 0.001, 0.8, 10)
      (S1, E1, I1, R1, Q1, L1), _r0r = simulate_quarantined_epidemic_euler(test_r0, 0.2, launch_date, 0.001, 0.8, 10)
      kwargs = {"legend_label" : "days of lockdown"} if launch_date == 20 else {}
      plot.line(t, 0.05 + I/N, line_color=color_mapper.palette[int((launch_date-20)*255/130.)], alpha=0.6)
      plot.line(t, I1/N, line_color=color_mapper.palette[int((launch_date-20)*255/130.)], alpha=0.6)
      plot.line(t, 0.1 * L1/365., line_color="red", alpha=0.2, **kwargs)
      plot2.line(t, Q1/N, line_color=color_mapper.palette[int((launch_date-20)*255/130.)], alpha=0.6)

    color_bar = bm.ColorBar(color_mapper=color_mapper, label_standoff=12, border_line_color=None, location=(0,0))
    plot.add_layout(color_bar, 'right')
    plot2.add_layout(color_bar, 'right')

    display(bl.row(plot, plot2))

In [0]:
iS, iE, iI, iR, iQ, iL = range(6)

from tqdm.notebook import tnrange
def run_simulations():
    # XXX refactor this to use numpy all the way since we know the array sizes
    results, lresults, details, r0rs = [], [], [], []
    for n in tnrange(samples, desc="Epidemic simulations:"):
        ret1, r0r1 = squee(r0_raw[n], 0, 0, lockdown_threshold[n], lockdown_r0[n], lockdown_limit[n])
        ret2, r0r2 = squee(r0_raw[n], cov_bt[n], app_launch_date[n], lockdown_threshold[n], lockdown_r0[n], lockdown_limit[n])
        ret3, r0r3 = squee(r0_raw[n], cov_gps[n], app_launch_date[n], lockdown_threshold[n], lockdown_r0[n], lockdown_limit[n])
        R1, Q1 = ret1[iR:iQ+1]
        R2, Q2 = ret2[iR:iQ+1]
        R3, Q3 = ret2[iR:iQ+1]
        base_fatality = np.round(infection_fatality_rate[n] * (R1[-1] + Q1[-1]))
        intervention_fatality = np.round(infection_fatality_rate[n] * (R2[-1] + Q2[-1]))
        intervention_gps_fatality = np.round(infection_fatality_rate[n] * (R3[-1] + Q3[-1]))
        base_lockdown = ret1[iL][-1]
        intervention_lockdown = ret2[iL][-1]
        intervention_gps_lockdown = ret3[iL][-1]
        lresults.append([intervention_lockdown, base_lockdown - intervention_lockdown, 
                         intervention_gps_lockdown, base_lockdown - intervention_gps_lockdown])
        results.append([intervention_fatality, base_fatality - intervention_fatality, 
                        intervention_gps_fatality, base_fatality - intervention_gps_fatality])
        details.append([ret1, ret2, ret3])
        r0rs.append([r0r1, r0r2, r0r3])
    results, lresults, details = map(np.array, [results, lresults, details])
    return results.T, lresults, details, r0rs


In [15]:
try:
    del results, lockdown_results, details, r0rs  # allow garbage collector to free up memory
except NameError: 
    pass
results, lockdown_results, details, r0rs = run_simulations()
# Saving results is both buggy (need to save more stuff) and does not persist across colab VMs :(
#reuse = True
#save = False
#if reuse:
#    results, lockdown_results, details, r0rs = np.load("simulations.npy", allow_pickle=True)
#else:
#    results, lockdown_results, details, r0rs = run_simulations()
#    if save:
#        np.save("simulations.npy", np.array((results, lockdown_results, details, r0rs)))

In [16]:
# plot state variables for some example runs from the monte carlo distribution
number = 20

def plot_variables(varspec, plot_range=range(number)):
    """
    Plot a set of simulation state variables (in side-by side charts to avoid clutter)

    varspec: [(varcolumn, "varname")]
    plot_range: which runs to plot
    """
    number = len(plot_range) // 2
    for var, name in varspec:
        top = np.clip(np.max(details[plot_range, 1, var]/N), 0, 1)
        kwargs = {"plot_width": 1024, "plot_height": 768, "y_axis_type": "linear" if var == iR else "log",
                  "y_range": [1/N, top + 0.01], # log compatible
                  "x_axis_label": "Day", "y_axis_label": "{0} proportion (lines) vs after app launch (dashed)".format(name),
                  "title": "{0} epidemics from the monte carlo distribution".format(number)}
        
        # produce figures in pairs to make them less crowded
        plot = bp.figure(**kwargs)
        kwargs["title"] = "{0} more epidemics from the monte carlo distribution".format(number)
        plot2 = bp.figure(**kwargs)
        plot.extra_y_ranges = plot2.extra_y_ranges = {"r0": bm.Range1d(start=0.5, end=4.5)}

        plot.add_layout(bm.LinearAxis(y_range_name="r0", axis_label="r0 (stripes)"), "right")
        plot2.add_layout(bm.LinearAxis(y_range_name="r0", axis_label="r0 (stripes)"), "right")

        color_mapper = bm.LinearColorMapper(palette="Turbo256", low=0, high=30)
        p = plot
        for i, n in enumerate(plot_range):
            ldate = app_launch_date[n]
            p.scatter(ldate, details[n, 0, var, ldate] / N, line_color="black", marker="x", legend_label="app launches")
            sub_n = i % number  # restart colours for the second graph
            col = color_mapper.palette[sub_n * (255 // (1 + number))]
            p.line(t, np.clip(details[n, 0, var]/N, -0.01, 1.0), color=col, alpha=0.8, legend_label=str(n))
            p.line(t[ldate:], np.clip(details[n, 1, var, ldate:]/N, -0.01, 1.0), color=col, alpha=0.8, line_dash="dashed")
            r0r = np.array(list(r0rs[n][1].items())).T
            p.scatter(r0r[0], r0r[1], color=col, alpha=0.2, y_range_name="r0")

            p = plot if i < (number - 1) else plot2
              
        plot.legend.background_fill_alpha = plot2.legend.background_fill_alpha = 0.85
        display(bl.row(plot, plot2))

plot_variables([(iE, "exposed"), (iI, "infected"), (iQ, "+ve app quarantined"), (iR, "recovered")])
#plot_variables([(iI, "infected")], [0, 1, 12, 13])

In [0]:
# which of our ODE simulators is numerically stable?
# Look for the most numerically pathological run in the batch

#x = np.argmin(details[:, 1, iR, -1])
test_solvers = False
if test_solvers:
    x = 41
    print(x, details[x, 1, iR, -1])
    I, R = details[x, 1, iI:iR+1]
    plot = bp.figure(plot_width=768, plot_height=400, y_axis_type="linear",
                    x_axis_label="Day", y_axis_label="Infected")

    #print((r0_raw[x], cov_bt[x], app_launch_date[x], lockdown_threshold[x], lockdown_r0[x]))
    t, I1, R1, L = simulate_quarantined_epidemic(r0_raw[x], cov_bt[x], app_launch_date[x], lockdown_threshold[x], lockdown_r0[x])
    (S, E, I, R, Q, L), _r0r = simulate_quarantined_epidemic2(r0_raw[x], cov_bt[x], app_launch_date[x], lockdown_threshold[x], lockdown_r0[x], lockdown_limit[x])
    (S2, E2, I2, R2, Q2, L), _r0r = simulate_quarantined_epidemic_euler(r0_raw[x], cov_bt[x], app_launch_date[x], lockdown_threshold[x], lockdown_r0[x], lockdown_limit[x])
    plot.line(t, I/N, legend_label="I (sqe2)", line_color="navy")
    plot.line(t, R/N, legend_label="R (sqe2)", line_color="red")
    #plot.line(t, I1/N, legend_label="I (sqe)", line_color="green")
    #plot.line(t, R1/N, legend_label="R (sqe)", line_color="blue")
    plot.line(t, I2/N, legend_label="I (squee)", line_color="purple")
    plot.line(t, R2/N, legend_label="R (squee)", line_color="orange")
    display(plot)
    print('Probe for numerical instability (the "squee" Euler simulator is looking okay)')

In [0]:
bin_05 = lambda data: bin_stats(data, lambda x: np.quantile(x, 0.05))
bin_95 = lambda data: bin_stats(data, lambda x: np.quantile(x, 0.95))
bin_q = lambda q, data: bin_stats(data, lambda x: np.quantile(x, q))

def graph_distribution(plot, variable, name, colors, xvar=incs, alpha=0.5, fill=True):
    cols = bm.LinearColorMapper(palette=colors).palette
    pot = bm.ColumnDataSource({
        "xvar": xvar, 
        "b05": bin_05(variable),
        "b25": bin_25(variable),
        "b75": bin_75(variable),
        "b95": bin_95(variable)
    })
    if fill:
        band = bm.Band(base="xvar", lower="b05", upper="b95",  line_width=1.6, source=pot, fill_color=cols[2], fill_alpha=alpha)
        plot.add_layout(band)
        band = bm.Band(base="xvar", lower="b25", upper="b75",  line_width=1.6, source=pot, fill_color=cols[1], fill_alpha=alpha)
        plot.add_layout(band)
    plot.line(xvar, bin_averages(variable), line_color=cols[0], legend_label="{0} (mean)".format(name), line_width=2.0)
    plot.line(xvar, bin_05(variable), line_color=cols[2], legend_label="{0} 5/95% quantiles".format(name), line_width=1.2)
    plot.line(xvar, bin_95(variable), line_color=cols[2], line_width=1.2)
    plot.line(xvar, bin_25(variable), line_color=cols[1], legend_label="{0} 25/75% quantiles".format(name), line_width=1.6)
    plot.line(xvar, bin_75(variable), line_color=cols[1], line_width=1.6)



In [19]:
plot = bp.figure(x_range=[0,1], plot_width=768, plot_height=768,
                 x_axis_label="Proportion of population using app",
                 y_axis_label="Mortality", y_axis_type="log")
intervention, diff, intervention_gps, gps_diff = results

graph_distribution(plot, intervention, "deaths", "OrRd4", alpha=0.2)

#plot.scatter(pop_adoption, gps_diff, radius=0.005, fill_alpha=0.2, fill_color="#500050", line_color=None,
#             legend_label="gps lives saved (1 simulation pair)")

plot2 = bp.figure(x_range=[0,1], plot_width=768, plot_height=768,
                 x_axis_label="Proportion of population using app",
                 y_axis_label="Mortality", y_axis_type="log")
graph_distribution(plot2, diff, "bluetooth lives saved", "Blues4", alpha=0.4)

plot2.scatter(pop_adoption, diff, radius=0.005, fill_alpha=0.2, fill_color="#50b050", line_color=None,
             legend_label="bt lives saved (1 simulation pair)")


#plot.line(incs, avgs, line_color="#a000f0", legend_label="prospective bluetooth matching",
#          line_width=2.0)  
plot.legend.background_fill_alpha = 0.75
display(bl.row(plot, plot2))

In [20]:
plot = bp.figure(x_range=[0,1], plot_width=768, plot_height=768,
                 x_axis_label="Proportion of population using app",
                 y_axis_label="Days of lockdown")

lintervention, ldiff, lintervention_gps, ldiff_gps = lockdown_results.T

graph_distribution(plot, lintervention, "lockdown days", "OrRd4")



graph_distribution(plot, ldiff, "bluetooth days averted", "Blues4")


plot.scatter(pop_adoption, ldiff, radius=0.005, fill_alpha=0.2, fill_color="#50b050", line_color=None,
             legend_label="lockdown days averted (1 simulation pair)")
plot.legend.background_fill_alpha = 0.95
plot.legend.location = "bottom_left"

#plot.line(incs, avgs, line_color="#a000f0", legend_label="prospective bluetooth matching",
#          line_width=2.0)  
display(plot)

In [21]:

def qbin_stats(variable, group_by):
    step = 1 / len(incs)
    quantiles = np.arange(step, 1, step)

    qstarts = np.quantile(group_by, quantiles)
    xyview = np.stack([group_by, variable], axis=-1)
    bin_indexes = np.searchsorted(qstarts, group_by)

    
    bins = [[] for i in incs]
    for i, val in enumerate(variable):
        bins[bin_indexes[i]].append(val)
    bins = list(map(np.array, bins))
    print(list(map(len, bins)))
    #hist =  np.histogramdd(xyview, [qstarts, 1])
    return qstarts, bins

for name, dist in data_raw.items():
    qstarts, bins = qbin_stats(diff, dist)
    plot = bp.figure(x_range=[np.min(dist), np.max(dist)], plot_width=768, plot_height=768,
                 x_axis_label=name,
                 y_axis_label="Lives saved")
    print(name)
    #print(qstarts, list(map(np.average, bins)))
    plot.line(qstarts, list(map(np.average, bins)))
    display(plot)

[295, 294, 294, 294, 294, 294, 294, 294, 294, 294, 294, 295, 294, 294, 294, 294, 294, 294, 294, 294, 294, 294, 295, 294, 294, 294, 294, 294, 294, 294, 294, 294, 294, 295]
pop_adoption


[295, 294, 294, 294, 294, 294, 294, 294, 294, 294, 294, 295, 294, 294, 294, 294, 294, 294, 294, 294, 294, 294, 295, 294, 294, 294, 294, 294, 294, 294, 294, 294, 294, 295]
tester_adoption


[295, 294, 294, 294, 294, 294, 294, 294, 294, 294, 294, 295, 294, 294, 294, 294, 294, 294, 294, 294, 294, 294, 295, 294, 294, 294, 294, 294, 294, 294, 294, 294, 294, 295]
testing_rate


[295, 294, 294, 294, 294, 294, 294, 294, 294, 294, 294, 295, 294, 294, 294, 294, 294, 294, 294, 294, 294, 294, 295, 294, 294, 294, 294, 294, 294, 294, 294, 294, 294, 295]
catch_rate_bt


[295, 294, 294, 294, 294, 294, 294, 294, 294, 294, 294, 295, 294, 294, 294, 294, 294, 294, 294, 294, 294, 294, 295, 294, 294, 294, 294, 294, 294, 294, 294, 294, 294, 295]
catch_rate_gps


[295, 294, 294, 294, 294, 294, 294, 294, 294, 294, 294, 295, 294, 294, 294, 294, 294, 294, 294, 294, 294, 294, 295, 294, 294, 294, 294, 294, 294, 294, 294, 294, 294, 295]
r0_raw


[379, 360, 400, 438, 0, 401, 428, 0, 413, 398, 378, 0, 405, 385, 389, 0, 420, 391, 422, 0, 392, 416, 0, 397, 397, 371, 0, 401, 401, 399, 0, 405, 414, 0]
app_launch_date


/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:393: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


ValueError: ignored

In [0]:
# Partial ranked correlation coefficient 

In [0]:
p = bp.figure(tools="", match_aspect=True, background_fill_color='black', y_range=[-1,1])
p.grid.visible = False
hexes=bh.hexbin(pop_adoption, diff/N, 0.005, aspect_scale=1/12.)
print(np.min(pop_adoption), np.max(pop_adoption))
p.hex_tile(q="q", r="r", size=0.1, line_color=None, source=hexes,
           fill_color=bt.linear_cmap('counts', 'Viridis256', 0, max(hexes.counts)))
#bi.show(p)

In [0]:
# try density maps. FIXME: these need to be normalised to not imply that most of the points lie on the
# left...
p = bp.figure(tools="", match_aspect=True, background_fill_color='black')
p.grid.visible = False

hexes=bh.hexbin(pop_adoption, cov_bt, 0.01)
p.hex_tile(q="q", r="r", size=0.1, line_color=None, source=hexes,
           fill_color=bt.linear_cmap('counts', 'Viridis256', 0, max(hexes.counts)/10))
bi.show(p)
p = bp.figure(tools="", match_aspect=True, background_fill_color='#440154')
p.grid.visible = False

hexes=bh.hexbin(pop_adoption, cov_gps, 0.01)
p.hex_tile(q="q", r="r", size=0.1, line_color=None, source=hexes,
           fill_color=bt.linear_cmap('counts', 'Viridis256', 0, max(hexes.counts)/10))
bi.show(p)

In [0]:
# Neither beta nor lognormal distributions fit Kucharski's
plot = bp.figure(y_axis_label="PDF", y_range=[0,1])
alt_ifr = npr.lognormal(-0.1, 0.9, size=samples).clip(0,8) / 100.

r0_dist = (1.5 + npr.beta(2, 5, size=samples) * 3)/100.

print ("mean", np.mean(r0_dist))


for name, dist, col in [("lognormal IFR", alt_ifr, "red"), ("beta IFR", infection_fatality_rate, "navy"),
                        ("r0", r0_raw/100, "green"), ("testing_rate", testing_rate/10, "orange")]:
    hist, edges = np.histogram(100 * dist, density=True, bins=100)
    plot.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
              fill_color=col, line_color="white", alpha=0.5, legend_label=name)
    
display(plot)

In [0]:
N